<a href="https://colab.research.google.com/github/chonorat/Classification/blob/master/Genome_Gene_Selection--Chi_Sqaure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.svm import LinearSVC
import sys
import random
from array import array

def read_data(datafile):
        with open(datafile) as file:
                lines = [i.strip() for i in file]
        data=[]
        for row_index in range(0,len(lines)):
                data.append([int(n) for n in lines[row_index].split()])
        return data

def chi_calc(observed_table):
    chi=0
    for classs in range(0,len(observed_table)):
            for obs_val in range(0,len(observed_table[0])):
                    exp_val=(observed_table[classs][obs_val]+observed_table[(classs-1)**2][obs_val])/2
                    try:
                        chi+=(observed_table[classs][obs_val]-exp_val)**2/exp_val
                    except: 
                        chi=chi
    return chi


##Select Features
def select_feature(top_x_features,data):
	features=[]
	for i in range(0,top_x_features):
		features.append(chi_table[i][0])
	##create feature data
	feature_data=[]
	for j in range(0,top_x_features):
		to_append=[]
		for i in range(0,len(data)):
			to_append.append(data[i][features[j]])
		feature_data.append(to_append)
	feature_data=list(zip(*feature_data))

	return feature_data,features

	
##Splitdata
def split_data(feature_data,trainlabels,data):
	train_size=round(.75*len(data))
	test_size=len(data)-train_size
	x_train,y_train,x_test,y_test=[],[],[],[]
	rand_sample=random.sample(range(0,(len(data)-1)), train_size)
	for sample_index in range(0,len(rand_sample)):
		x_train.append(feature_data[rand_sample[sample_index]])
		y_train.append(trainlabels[rand_sample[sample_index]])
	for row_index in range(0,len(data)):
		if row_index not in rand_sample:
			x_test.append(feature_data[row_index])
			y_test.append(trainlabels[row_index])
	return x_train,y_train,x_test,y_test


##Main Methods
testdata=read_data(sys.argv[3])#"/Users/chono/OneDrive/Desktop/Summer_19/ml/testdata")
labels=read_data(sys.argv[2])#"/Users/chono/OneDrive/Desktop/Summer_19/ml/labels")
data=read_data(sys.argv[1])#"/Users/chono/Downloads/DNX_MGRIP_GPD-WIN2/traindata (4)/traindata")
chi_table=[]
for column_index in range(0,len(data[0])):
        observed_table=[[0,0,0],[0,0,0]]
        for row_index in range(0,len(data)):
                observed_table[labels[row_index][0]][data[row_index][column_index]]+=1
        chi=chi_calc(observed_table)
        chi_table.append([column_index,chi])	
chi_table.sort(key=lambda x:x[1],reverse=True)

##Select labels
trainlabels=[]
for i in range(0,len(data)):
	trainlabels.append(labels[i][0])


##Predict Test Data
feature_data,top_features=select_feature(15,data)
score=0
clf = LinearSVC(random_state=0, tol=1e-5,max_iter=10000,C=1.25)
for valid_runs in range(0,10):
	x_train,y_train,x_test,y_test=split_data(feature_data,trainlabels,data)	
	clf.fit(x_train, y_train)
	score+=clf.score(x_test,y_test)
	print(clf.score(x_test,y_test))
print("avg_score=",score/10)
clf.fit(feature_data, trainlabels)
test_data,test_features=select_feature(15,testdata)
predictions=list(clf.predict(test_data))
print(predictions)



